In [1]:
from flask import Flask , request , jsonify
import pyodbc
import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId
import threading
import time
from datetime import datetime, date
from decimal import Decimal
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import requests

In [2]:
def connect_db():
    conn_str = (
        'DRIVER={ODBC Driver 17 for SQL Server};'
        'SERVER=localhost\\SQLEXPRESS;'
        'DATABASE=SalesDB;'
        'Trusted_Connection=yes;'
    )
    return pyodbc.connect(conn_str)

In [3]:
def get_mongo_collection():
    client = MongoClient("mongodb://localhost:27017/")  # adjust if needed
    db = client["MyDatabase"]  # database name
    return db["RetailCustomerData_Coll"]  # collection name

In [4]:
app = Flask(__name__)

In [5]:
# ---------- Sync Endpoint ----------
@app.route('/sync-to-mongo', methods=['POST'])
def sync_to_mongo():
    try:
        join_query = """
        SELECT 
            c.CustomerID,
            c.Name AS CustomerName,
            c.Gender,
            c.BirthDate,
            c.City AS CustomerCity,
            b.BranchID,
            b.BranchName,
            b.City AS BranchCity,
            s.SaleID,
            s.SaleDate,
            s.TotalAmount,
            p.ProductID,
            p.ProductName,
            p.Category,
            sd.Quantity,
            sd.UnitPrice,
            (sd.Quantity * sd.UnitPrice) AS TotalPrice
        FROM Customers c
        JOIN Sales s ON c.CustomerID = s.CustomerID
        JOIN Branches b ON s.BranchID = b.BranchID
        JOIN SaleDetails sd ON s.SaleID = sd.SaleID
        JOIN Products p ON sd.ProductID = p.ProductID
        """

        # Get SQL data
        conn = connect_db()
        df = pd.read_sql(join_query, conn)
        conn.close()

        # 🔹 Convert BirthDate & SaleDate to datetime.datetime objects
        for col in ["BirthDate", "SaleDate"]:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col], errors="coerce")  # handles invalid dates

        # Convert DataFrame to list of dicts
        data_records = df.to_dict(orient="records")

        # Insert into MongoDB
        col = get_mongo_collection()
        col.delete_many({})  # Clear old data
        if data_records:
            col.insert_many(data_records)

        return jsonify({
            "message": f"Synced {len(data_records)} records into MongoDB collection 'retail_data' with proper datetime fields"
        }), 201

    except Exception as e:
        return jsonify({"error": str(e)}), 500

In [6]:
# Code to Run the Flask API 
def run_flask():
    app.run(port=5000, debug=False, use_reloader=False)

# Code to Run the Flask API in a separate thread in the background  
threading.Thread(target=run_flask).start()
time.sleep(1)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


C:\Users\Soft LapTop\AppData\Local\Temp\ipykernel_25956\4217954131.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(join_query, conn)
127.0.0.1 - - [27/Aug/2025 23:54:36] "POST /sync-to-mongo HTTP/1.1" 201 -
127.0.0.1 - - [27/Aug/2025 23:54:51] "POST /generate_sales_collection HTTP/1.1" 404 -


In [7]:
# Make POST request to sync SQL Server -> MongoDB
sync_resp = requests.post("http://127.0.0.1:5000/sync-to-mongo")

# Print the status and the JSON response
print("Status Code:", sync_resp.status_code)
print("Response JSON:", sync_resp.json())

Status Code: 201
Response JSON: {'message': "Synced 1496 records into MongoDB collection 'retail_data' with proper datetime fields"}
